In [1]:
import cv2
import os
import imutils
import mediapipe as mp

personName = 'Hola'
dataPath = 'D:\\5toSemestre\\Proce Digital Imagenes\\ProyectosFinales\\SistemaBiometrico\\Fotos'

personPath = os.path.join(dataPath, personName)

if not os.path.exists(personPath):
    print('Carpeta creada:', personPath)
    os.makedirs(personPath)

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# cap = cv2.VideoCapture('Video.mp4')

count = 0

# Inicializa MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = imutils.resize(frame, width=640)
    auxFrame = frame.copy()

    # Convertir la imagen a RGB para MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Dibujar la malla facial
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec)

            # Obtener las coordenadas de la caja delimitadora del rostro
            h, w, _ = frame.shape
            face_coords = [(int(point.x * w), int(point.y * h)) for point in face_landmarks.landmark]
            x_min = min([coord[0] for coord in face_coords])
            y_min = min([coord[1] for coord in face_coords])
            x_max = max([coord[0] for coord in face_coords])
            y_max = max([coord[1] for coord in face_coords])

            # Extraer y guardar el rostro
            rostro = auxFrame[y_min:y_max, x_min:x_max]
            if rostro.size != 0:
                rostro = cv2.resize(rostro, (250, 250), interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(os.path.join(personPath, f'rostro_{count}.jpg'), rostro)
                count += 1

    # Mostrar la cantidad de fotos tomadas en la pantalla
    cv2.putText(frame, "Fotos tomadas: {}".format(count), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow('MediaPipe Face Mesh', frame)

    k = cv2.waitKey(1)
    if k == 27 or count >= 300:
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\henry\miniconda3\envs\env-PrimerEntorno\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
